In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import kapre
import keras
import os
import soundfile as sf
import time
import progressbar
import gc
import h5py

Using TensorFlow backend.


In [2]:
# Load the training and validation data from hdf5
training = h5py.File('/data/musicnet/hdf5/train_data.hdf5', 'r')
validation = h5py.File('/data/musicnet/hdf5/val_data.hdf5', 'r')

list(training.keys())
training_data = training['data']
training_targets = training['targets']

list(validation.keys())
validation_data = validation['data']
validation_targets = validation['targets']

In [ ]:
# LSTM AUTOENCODER

In [3]:
# Reshaping the data into sequences
# The shape needs to be (number of sample, number of timesteps, number of features)
# In this case we have a sequence of frequencies, i.e. just one feature, with 88200 (sampling rate of 44100 per
# second over 2 second chunks)
train_data = training_data.value.reshape(training_data.shape[0],88200,1)
val_data = validation_data.value.reshape(validation_data.shape[0],88200,1)
print(train_data.shape)
print(val_data.shape)

(39472, 88200, 1)
(8089, 88200, 1)


In [29]:
# Defining the model
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model

inputs = Input(shape=(88200, 1))
encoded = LSTM(128)(inputs)

decoded = RepeatVector(88200)(encoded)
decoded = LSTM(1, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

In [30]:
sequence_autoencoder.compile(loss='mean_squared_error', optimizer='adam')  # original

In [32]:
sequence_autoencoder.fit(train_data[:20], train_data[:20],
                     epochs=1,
                     batch_size=10,
                     shuffle=True,
                     )

Epoch 1/1
20/20 [==============================] - 267s 13s/step - loss: 0.0052


In [50]:
sequence_autoencoder.layers[1].get_weights()[1]

array([[-0.04869734,  0.04266794, -0.05007424, ...,  0.04580592,
         0.04331943,  0.01576233],
       [ 0.02189239, -0.01102153, -0.04808544, ..., -0.08176541,
         0.07128568, -0.01161315],
       [ 0.02004603, -0.00435795, -0.06273201, ...,  0.01545435,
         0.01739861,  0.02938924],
       ...,
       [-0.06072284,  0.0325784 , -0.02698366, ...,  0.00060323,
         0.01385953, -0.02302717],
       [ 0.03626321, -0.04324282, -0.01008554, ...,  0.00514694,
         0.00817982, -0.03254625],
       [ 0.02788613, -0.03105488, -0.01109948, ..., -0.05314592,
        -0.01351892, -0.02408768]], dtype=float32)

In [53]:
train_data[:20]

array([[[-0.00576782],
        [-0.00973511],
        [-0.01086426],
        ...,
        [ 0.00796509],
        [ 0.00842285],
        [ 0.00869751]],

       [[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        ...,
        [ 0.02810669],
        [ 0.02606201],
        [ 0.02261353]],

       [[-0.00878906],
        [-0.0112915 ],
        [-0.01251221],
        ...,
        [-0.06298828],
        [-0.06079102],
        [-0.05648804]],

       ...,

       [[ 0.04406738],
        [ 0.04138184],
        [ 0.0385437 ],
        ...,
        [ 0.01885986],
        [ 0.02087402],
        [ 0.02209473]],

       [[-0.02990723],
        [-0.04486084],
        [-0.05502319],
        ...,
        [ 0.0032959 ],
        [ 0.00311279],
        [ 0.00302124]],

       [[-0.11203003],
        [-0.10736084],
        [-0.09985352],
        ...,
        [-0.06072998],
        [-0.06280518],
        [-0.06814575]]])